# 02 – Preprocessing & Feature Engineering
Bu notebook, Loan Prediction projesinde modelleme aşamasına geçmeden önce yapılan tüm veri hazırlama adımlarını içerir. Aşağıdaki işlemler sistematik şekilde uygulanmıştır:
* Ham verinin temizlenmesi ve tür dönüşümleri
* Değişken sınıflandırması (kategorik, sayısal, flag, num-but-cat, cat-but-car)
* Eksik değer işlemleri
    * Kategorik → Mode imputasyonu
    * Sayısal → StandardScaler + KNN Imputer
    * Eksik veri flag'leri
* Aykırı değer tespiti ve IQR clipping
* Feature Engineering
    * Domain tabanlı yeni değişken üretimi
    * Oranlar, dönüşümler, log-transformlar
* Encoding & Scaling (ColumnTransformer ile)
* Feature Selection
    * Mutual Information

Bu notebook’un çıktıları, modelleme aşamasında (03_modelling.ipynb) doğrudan kullanılmak üzere kayıt altına alınmaktadır.

## 1) VERİ HAZIRLIĞI (RAW → CLEANED)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


# 1) Ham Veriyi Yükle
train_raw = pd.read_csv("../data/raw/train_u6lujuX_CVtuZ9i.csv")
test_raw  = pd.read_csv("../data/raw/test_Y3wMUE5_7gLdaTN.csv")

# Çalışma kopyaları (ham veri bozulmasın)
data = train_raw.copy()
data_test = test_raw.copy()

print("Train shape:", data.shape)
print("Test shape: ", data_test.shape)



# 2) ID ve Hedef Ayrımı

# Hedef değişkeni 0/1 olarak dönüştür
data["Loan_Status"] = data["Loan_Status"].map({"Y": 1, "N": 0})

TARGET = "Loan_Status"

# Loan_ID modelde kullanılmayacak → hem train hem testten çıkar
if "Loan_ID" in data.columns:
    data.drop(columns=["Loan_ID"], inplace=True)

if "Loan_ID" in data_test.columns:
    data_test.drop(columns=["Loan_ID"], inplace=True)


# 3) Veri Tipi Düzeltmeleri

# Numerik coercion yapılacak sütunlar
num_cols_to_coerce = [
    "ApplicantIncome",
    "CoapplicantIncome",
    "LoanAmount",
    "Loan_Amount_Term",
    "Credit_History"
]

for col in num_cols_to_coerce:
    if col in data.columns:
        data[col] = pd.to_numeric(data[col], errors="coerce")
    if col in data_test.columns:
        data_test[col] = pd.to_numeric(data_test[col], errors="coerce")

if "Dependents" in data.columns:
    data["Dependents"] = data["Dependents"].replace("3+", "3")
    data["Dependents"] = pd.to_numeric(data["Dependents"], errors="coerce")

if "Dependents" in data_test.columns:
    data_test["Dependents"] = data_test["Dependents"].replace("3+", "3")
    data_test["Dependents"] = pd.to_numeric(data_test["Dependents"], errors="coerce")


# 4) Eksik Değer Flag Kolonları (Leakage yok → split öncesi yapılır)

for col in data.columns:
    if data[col].isnull().any():
        flag_name = f"is_missing_{col}"
        data[flag_name] = data[col].isnull().astype(int)

for col in data_test.columns:
    if data_test[col].isnull().any():
        flag_name = f"is_missing_{col}"
        data_test[flag_name] = data_test[col].isnull().astype(int)


print("\nEksik değer (train):")
print(data.isnull().sum())

print("\nEksik değer (test):")
print(data_test.isnull().sum())


# 5) Train / Validation Ayrımı

X_full = data.drop(columns=[TARGET])
y_full = data[TARGET]

X_train_raw, X_val_raw, y_train, y_val = train_test_split(
    X_full,
    y_full,
    test_size=0.2,
    random_state=42,
    stratify=y_full
)

X_test_raw = data_test.copy()

print("\nTrain:", X_train_raw.shape)
print("Validation:", X_val_raw.shape)
print("Test:", X_test_raw.shape)

print("\n✓ Veri Hazırlığı Aşaması Tamamlandı.")


Train shape: (614, 13)
Test shape:  (367, 12)

Eksik değer (train):
Gender                         13
Married                         3
Dependents                     15
Education                       0
Self_Employed                  32
ApplicantIncome                 0
CoapplicantIncome               0
LoanAmount                     22
Loan_Amount_Term               14
Credit_History                 50
Property_Area                   0
Loan_Status                     0
is_missing_Gender               0
is_missing_Married              0
is_missing_Dependents           0
is_missing_Self_Employed        0
is_missing_LoanAmount           0
is_missing_Loan_Amount_Term     0
is_missing_Credit_History       0
dtype: int64

Eksik değer (test):
Gender                         11
Married                         0
Dependents                     10
Education                       0
Self_Employed                  23
ApplicantIncome                 0
CoapplicantIncome               0
LoanAmount    

## 2) DEĞİŞKEN SINIFLANDIRMASI

In [2]:
from pandas.api.types import (
    is_object_dtype,
    is_numeric_dtype,
    is_bool_dtype,
    is_datetime64_any_dtype
)

def grab_col_names(dataframe, cat_th=10, car_th=20):


    cols = dataframe.columns

    # Datetime kolonları
    datetime_cols = [c for c in cols if is_datetime64_any_dtype(dataframe[c])]

    # 1) Kategorik kolonlar (object veya bool)
    cat_cols = [
        c for c in cols
        if (is_object_dtype(dataframe[c]) or is_bool_dtype(dataframe[c]))
        and c not in datetime_cols
    ]

    # 2) Numerik ama az unique değerli kolonlar (num_but_cat)
    num_but_cat = [
        c for c in cols
        if is_numeric_dtype(dataframe[c])
        and dataframe[c].nunique(dropna=True) < cat_th
    ]

    # 3) Kategorik görünümlü ama yüksek kardinalite (cat_but_car)
    cat_but_car = [
        c for c in cat_cols
        if dataframe[c].nunique(dropna=True) > car_th
    ]

    # 4) Güncellenmiş kategorik kolonlar
    cat_cols = cat_cols + num_but_cat
    cat_cols = [c for c in cat_cols if c not in cat_but_car]

    # 5) Numerik kolonlar (num_but_cat hariç)
    num_cols = [
        c for c in cols
        if is_numeric_dtype(dataframe[c])
        and c not in num_but_cat
    ]

    print("====== DEĞİŞKEN TÜR ÖZETİ (TRAIN) ======")
    print(f"Observation (satır):          {dataframe.shape[0]}")
    print(f"Variables (sütun):            {dataframe.shape[1]}")
    print(f"Cat cols:                     {len(cat_cols)}")
    print(f"Num cols:                     {len(num_cols)}")
    print(f"Cat but car (yüksek kard.):   {len(cat_but_car)}")
    print(f"Num but cat:                  {len(num_but_cat)}")
    print(f"Datetime cols:                {len(datetime_cols)}")

    return cat_cols, num_cols, cat_but_car, datetime_cols, num_but_cat


# Sadece TRAIN üzerinden tür sınıflandırma 
cat_cols, num_cols, cat_but_car, datetime_cols, num_but_cat = grab_col_names(X_train_raw)

# Loan_ID daha önce drop edildi, burada olmamalı. 
# Yine de kontrol edelim (güvenlik için):
for c in ["Loan_ID", "Loan_Status"]:
    if c in cat_cols:
        cat_cols.remove(c)
    if c in num_cols:
        num_cols.remove(c)

# Ek yardımcı listeler (modelleme ve preprocessing için)

# Eksik veri bayrak kolonları (A aşamasında oluşturmuştuk: is_missing_*)
missing_flag_cols = [c for c in X_train_raw.columns if c.startswith("is_missing_")]

# Binary (2 sınıflı) numerik kolonlar
binary_num_cols = [
    c for c in num_cols
    if X_train_raw[c].nunique(dropna=True) == 2 and c not in missing_flag_cols
]

# Geriye kalan “gerçek” sürekli numerikler
num_cont_cols = [
    c for c in num_cols
    if c not in binary_num_cols and c not in missing_flag_cols
]

print("\n--- Liste Detayları ---")
print("Kategorik kolonlar (cat_cols):")
print(cat_cols)

print("\nSürekli numerik kolonlar (num_cont_cols):")
print(num_cont_cols)

print("\nBinary numerik kolonlar (binary_num_cols):")
print(binary_num_cols)

print("\nEksik flag kolonları (missing_flag_cols):")
print(missing_flag_cols)


====== DEĞİŞKEN TÜR ÖZETİ (TRAIN) ======
Observation (satır):          491
Variables (sütun):            18
Cat cols:                     14
Num cols:                     4
Cat but car (yüksek kard.):   0
Num but cat:                  9
Datetime cols:                0

--- Liste Detayları ---
Kategorik kolonlar (cat_cols):
['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area', 'Dependents', 'Credit_History', 'is_missing_Gender', 'is_missing_Married', 'is_missing_Dependents', 'is_missing_Self_Employed', 'is_missing_LoanAmount', 'is_missing_Loan_Amount_Term', 'is_missing_Credit_History']

Sürekli numerik kolonlar (num_cont_cols):
['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']

Binary numerik kolonlar (binary_num_cols):
[]

Eksik flag kolonları (missing_flag_cols):
['is_missing_Gender', 'is_missing_Married', 'is_missing_Dependents', 'is_missing_Self_Employed', 'is_missing_LoanAmount', 'is_missing_Loan_Amount_Term', 'is_missing_Credit_History']


## 3) EKSİK DEĞER İŞLEME (MISSING VALUE HANDLING)
#### - Kategorik: Mode (most_frequent)
#### - Sayısal: StandardScaler + KNNImputer

In [3]:
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler

# 1) Kategorik / Sayısal / Flag kolonlarını netleştir
cat_cols_no_flags = [c for c in cat_cols if c not in missing_flag_cols]

print("Kategorik (impute edilecek) kolonlar:", cat_cols_no_flags)
print("Sayısal (KNN ile impute edilecek) kolonlar:", num_cont_cols)
print("Flag kolonları (dokunulmayacak):", missing_flag_cols)

# 2) KATEGORİK EKSİK DEĞERLER (MODE)

cat_imputer = SimpleImputer(strategy="most_frequent")
cat_imputer.fit(X_train_raw[cat_cols_no_flags]) 

def impute_categoricals(df):
    df_cat = pd.DataFrame(
        cat_imputer.transform(df[cat_cols_no_flags]),
        columns=cat_cols_no_flags,
        index=df.index
    )
    return df_cat

X_train_cat_imp = impute_categoricals(X_train_raw)
X_val_cat_imp   = impute_categoricals(X_val_raw)
X_test_cat_imp  = impute_categoricals(X_test_raw)

# Flag kolonları doğrudan alınacak (0/1, eksik yok)
def get_flags(df):
    # Sadece df'de gerçekten var olan flag kolonlarını al
    cols = [c for c in missing_flag_cols if c in df.columns]
    return df[cols].copy() if cols else pd.DataFrame(index=df.index)

X_train_flags = get_flags(X_train_raw)
X_val_flags   = get_flags(X_val_raw)
X_test_flags  = get_flags(X_test_raw)

# 3) SAYISAL EKSİK DEĞERLER (StandardScaler + KNNImputer)

# Train sayısal verisi
num_train = X_train_raw[num_cont_cols].copy()

# a) Train ortalamaları
train_means = num_train.mean()

# b) StandardScaler: önce NaN'leri ortalamayla doldur, sonra scale et
scaler = StandardScaler()
num_train_scaled = pd.DataFrame(
    scaler.fit_transform(num_train.fillna(train_means)),
    columns=num_cont_cols,
    index=num_train.index
)

# c) KNN doldurması için NaN pozisyonlarını geri getir
for c in num_cont_cols:
    num_train_scaled.loc[num_train[c].isna(), c] = np.nan

# d) KNNImputer'ı sadece train'de fit et
knn = KNNImputer(n_neighbors=5, weights="uniform")
num_train_imp_scaled = pd.DataFrame(
    knn.fit_transform(num_train_scaled),
    columns=num_cont_cols,
    index=num_train_scaled.index
)

# e) Orijinal skala
num_train_imp = pd.DataFrame(
    scaler.inverse_transform(num_train_imp_scaled),
    columns=num_cont_cols,
    index=num_train_imp_scaled.index
)

print("\nNumerik impute sonrası (train) NaN sayıları:")
print(num_train_imp.isnull().sum())

# Validation & Test için aynı süreç (fit YOK, sadece transform)

def impute_numerics(df):
    num_part = df[num_cont_cols].copy()

    # 1) Train ortalamalarıyla doldur → scale
    num_scaled = pd.DataFrame(
        scaler.transform(num_part.fillna(train_means)),
        columns=num_cont_cols,
        index=num_part.index
    )

    # 2) NaN pozisyonlarını geri koy
    for c in num_cont_cols:
        num_scaled.loc[num_part[c].isna(), c] = np.nan

    # 3) KNN (train'de fit edilmiş)
    num_imp_scaled = pd.DataFrame(
        knn.transform(num_scaled),
        columns=num_cont_cols,
        index=num_part.index
    )

    # 4) Orijinal skala
    num_imp = pd.DataFrame(
        scaler.inverse_transform(num_imp_scaled),
        columns=num_cont_cols,
        index=num_part.index
    )

    return num_imp

X_val_num_imp  = impute_numerics(X_val_raw)
X_test_num_imp = impute_numerics(X_test_raw)

print("\nNumerik impute sonrası (val) NaN sayıları:")
print(X_val_num_imp.isnull().sum())
print("\nNumerik impute sonrası (test) NaN sayıları:")
print(X_test_num_imp.isnull().sum())

# 4) PARÇALARI BİRLEŞTİR (NUM + CAT + FLAGS + DİĞER)

def rebuild_imputed_df(df_raw, num_imp, cat_imp, flags):
    parts = [num_imp, cat_imp]

    # Flag kolonları: sadece df_raw içinde gerçekten olanlar
    if not flags.empty:
        parts.append(flags)

    # Num + cat + flag dışında kalan kolonları da ekle (varsa)
    used_cols = num_cont_cols + cat_cols_no_flags + [c for c in flags.columns]
    other_cols = [c for c in df_raw.columns if c not in used_cols]

    if other_cols:
        parts.append(df_raw[other_cols])

    df_new = pd.concat(parts, axis=1)
    return df_new

train_imputed = rebuild_imputed_df(X_train_raw, X_train_num_imp := num_train_imp, X_train_cat_imp, X_train_flags)
val_imputed   = rebuild_imputed_df(X_val_raw,   X_val_num_imp,   X_val_cat_imp,   X_val_flags)
test_imputed  = rebuild_imputed_df(X_test_raw,  X_test_num_imp,  X_test_cat_imp,  X_test_flags)

print("\nEksik değerler - KNN + Mode sonrası (TRAIN):")
print(train_imputed.isnull().sum())

print("\nŞekiller (impute sonrası):")
print("train_imputed:", train_imputed.shape)
print("val_imputed:  ", val_imputed.shape)
print("test_imputed: ", test_imputed.shape)

print("\n✓ C) Eksik Değer İşleme Aşaması Tamamlandı.")


Kategorik (impute edilecek) kolonlar: ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area', 'Dependents', 'Credit_History']
Sayısal (KNN ile impute edilecek) kolonlar: ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
Flag kolonları (dokunulmayacak): ['is_missing_Gender', 'is_missing_Married', 'is_missing_Dependents', 'is_missing_Self_Employed', 'is_missing_LoanAmount', 'is_missing_Loan_Amount_Term', 'is_missing_Credit_History']

Numerik impute sonrası (train) NaN sayıları:
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
dtype: int64

Numerik impute sonrası (val) NaN sayıları:
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
dtype: int64

Numerik impute sonrası (test) NaN sayıları:
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
dtype: int64

Eksik değerler - KNN + Mode sonrası (TRAIN):
ApplicantIncome                

## 4) AYKIRI DEĞER İŞLEME (IQR CLIPPING)

In [4]:
# D) AYKIRI DEĞER İŞLEME (IQR CLIPPING)

import numpy as np
import pandas as pd

# Sadece numerik değişkenlerde clipping yapılır
numeric_cols = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']

def iqr_bounds(series):
    """Bir kolon için IQR tabanlı alt ve üst sınırları hesaplar."""
    q1, q3 = series.quantile(0.25), series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return lower, upper


# 1) TRAIN üzerinde IQR sınırlarını hesapla
bounds = {}
outlier_info = []

for col in numeric_cols:
    lower, upper = iqr_bounds(train_imputed[col])
    bounds[col] = (lower, upper)

    # Train içindeki aykırı değer sayısını hesapla
    outlier_count = ((train_imputed[col] < lower) | (train_imputed[col] > upper)).sum()
    outlier_info.append([col, outlier_count, lower, upper])


# Aykırı değer özet tablosu
outlier_df = pd.DataFrame(
    outlier_info,
    columns=["Feature", "Outlier Count (Train)", "Lower Bound", "Upper Bound"]
)

print("=== AYKIRI DEĞER BİLGİSİ (IQR - SADECE TRAIN'E GÖRE) ===")
print(outlier_df)


# 2) TRAIN / VAL / TEST için clipping fonksiyonu
def apply_clipping(df, bounds):
    df_clipped = df.copy()
    for col in numeric_cols:
        lower, upper = bounds[col]
        df_clipped[col] = df_clipped[col].clip(lower=lower, upper=upper)
    return df_clipped


# 3) IQR clipping uygula
train_final = apply_clipping(train_imputed, bounds)
val_final   = apply_clipping(val_imputed, bounds)
test_final  = apply_clipping(test_imputed, bounds)

print("\n✓ IQR Clipping tüm setlere uygulandı.")
print("train_final:", train_final.shape)
print("val_final:  ", val_final.shape)
print("test_final: ", test_final.shape)

# Hedef vektörlerini kopyala (Final)
y_train_final = y_train.copy()
y_val_final   = y_val.copy()

# 4) (OPSİYONEL) Clipping öncesi-sonrası istatistik karşılaştırması
print("\n=== İSTATİSTİK KARŞILAŞTIRMASI (Öncesi vs Sonrası) ===")
print(">>> Önce (train_imputed):")
print(train_imputed[numeric_cols].describe().T)

print("\n>>> Sonra (train_final):")
print(train_final[numeric_cols].describe().T)


=== AYKIRI DEĞER BİLGİSİ (IQR - SADECE TRAIN'E GÖRE) ===
             Feature  Outlier Count (Train)  Lower Bound  Upper Bound
0    ApplicantIncome                     40     -1472.50     10203.50
1  CoapplicantIncome                     21     -3361.50      5602.50
2         LoanAmount                     32        -1.25       268.75
3   Loan_Amount_Term                     74       360.00       360.00

✓ IQR Clipping tüm setlere uygulandı.
train_final: (491, 18)
val_final:   (123, 18)
test_final:  (367, 17)

=== İSTATİSTİK KARŞILAŞTIRMASI (Öncesi vs Sonrası) ===
>>> Önce (train_imputed):
                   count         mean          std    min     25%     50%  \
ApplicantIncome    491.0  5529.997963  6457.784318  210.0  2906.0  3859.0   
CoapplicantIncome  491.0  1569.537271  2789.523475    0.0     0.0  1032.0   
LoanAmount         491.0   147.227291    86.497217    9.0   100.0   128.0   
Loan_Amount_Term   491.0   341.474542    65.110838   12.0   360.0   360.0   

                 

## 5) FEATURE ENGINEERING (Yeni Özellik Üretimi)

In [5]:
train_fe = train_final.copy()
val_fe   = val_final.copy()
test_fe  = test_final.copy()

# 1) Domain Tabanlı Özellikler

def add_domain_features(df):
    
    # Toplam gelir
    df["Total_Income"] = df["ApplicantIncome"] + df["CoapplicantIncome"]

    # EMI: LoanAmount / Loan_Term
    term_safe = df["Loan_Amount_Term"].replace(0, np.nan)
    df["EMI"] = df["LoanAmount"] / term_safe

    # Kredi başına gelir
    amount_safe = df["LoanAmount"].replace(0, np.nan)
    df["Income_per_Loan"] = df["Total_Income"] / amount_safe

    # Loan term (yıl)
    df["Loan_Term_Years"] = df["Loan_Amount_Term"] / 12

    # Log dönüşümleri (stabil)
    df["Log_Total_Income"] = np.log1p(df["Total_Income"].clip(lower=0))
    df["Log_LoanAmount"]   = np.log1p(df["LoanAmount"].clip(lower=0))

    # Bağımlı kişi sayısı yüksek mi?
    df["Many_Dependents"] = (df["Dependents"] >= 3).astype(int)

    return df

train_fe = add_domain_features(train_fe)
val_fe   = add_domain_features(val_fe)
test_fe  = add_domain_features(test_fe)

# 2) Interaction Özellikleri

def add_interaction_features(df, eps=1e-6):
    # Eş gelir oranı
    df["Coapplicant_Income_Ratio"] = df["CoapplicantIncome"] / (df["Total_Income"] + eps)

    # Gelir / EMI = ödeme gücü göstergesi
    df["Income_to_EMI"] = df["Total_Income"] / (df["EMI"] + eps)

    # Kişi başına gelir
    dep_safe = (df["Dependents"] + 1).replace(0, np.nan)
    df["Per_Capita_Income"] = df["Total_Income"] / dep_safe

    return df

train_fe = add_interaction_features(train_fe)
val_fe   = add_interaction_features(val_fe)
test_fe  = add_interaction_features(test_fe)

# 2.5) Temizlik (NaN Trap Fix)
# Feature Engineering sırasında (özellikle bölme işlemlerinde)
# oluşabilecek sonsuz (inf) ve eksik (NaN) değerleri temizliyoruz.

def clean_new_features(df):
    # Sonsuz değerleri NaN yap
    df = df.replace([np.inf, -np.inf], np.nan)
    # NaN değerleri 0 ile doldur (yeni türetilenler için 0 güvenli bir varsayımdır)
    df = df.fillna(0)
    return df

train_fe = clean_new_features(train_fe)
val_fe   = clean_new_features(val_fe)
test_fe  = clean_new_features(test_fe)

# 3) Binning (Train’e göre) – quantile cut

# Sadece train üzerinde sınırlar
income_quantiles = train_fe["Total_Income"].quantile([0, 0.25, 0.5, 0.75, 1]).values
loan_quantiles   = train_fe["LoanAmount"].quantile([0, 0.25, 0.5, 0.75, 1]).values

# Unique check (aynı değerler varsa bin hatası olmasın diye)
income_bins = np.unique(income_quantiles)
loan_bins   = np.unique(loan_quantiles)

income_labels = [f"q{i}" for i in range(1, len(income_bins))]
loan_labels   = [f"q{i}" for i in range(1, len(loan_bins))]

def apply_binning(df):
    df["Total_Income_bin"] = pd.cut(
        df["Total_Income"],
        bins=income_bins,
        labels=income_labels,
        include_lowest=True,
        duplicates="drop"
    )

    df["LoanAmount_bin"] = pd.cut(
        df["LoanAmount"],
        bins=loan_bins,
        labels=loan_labels,
        include_lowest=True,
        duplicates="drop"
    )
    return df

train_fe = apply_binning(train_fe)
val_fe   = apply_binning(val_fe)
test_fe  = apply_binning(test_fe)

# 4) Final FE X/Y Matrisleri (Scaling/Encoding için hazır)

X_train_fe = train_fe.copy()
X_val_fe   = val_fe.copy()
X_test_fe  = test_fe.copy()

y_train_fe = y_train_final.copy()
y_val_fe   = y_val_final.copy()

print("Feature Engineering tamamlandı.")
print("X_train_fe shape:", X_train_fe.shape)
print("X_val_fe shape:  ", X_val_fe.shape)
print("X_test_fe shape: ", X_test_fe.shape)

Feature Engineering tamamlandı.
X_train_fe shape: (491, 30)
X_val_fe shape:   (123, 30)
X_test_fe shape:  (367, 29)


C:\Users\murat\AppData\Local\Temp\ipykernel_18288\3985671768.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([np.inf, -np.inf], np.nan)


## 6) ENCODING + SCALING (ColumnTransformer ile)

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler

TARGET = "Loan_Status"  # zaten X'lerde yok ama netlik için

# 1) X / y ayrımı (FE sonrası)
X_train_fe = train_fe.copy()
X_val_fe   = val_fe.copy()
X_test_fe  = test_fe.copy()   # test'te hedef yok

y_train_fe = y_train_final.copy()
y_val_fe   = y_val_final.copy()

# 1.1. Kolon hizalama (train → val/test)

# Val için eksik kolonları ekle
for col in X_train_fe.columns:
    if col not in X_val_fe.columns:
        X_val_fe[col] = 0

# Test için eksik kolonları ekle
for col in X_train_fe.columns:
    if col not in X_test_fe.columns:
        X_test_fe[col] = 0

# Eğer val/test içinde train'de olmayan fazladan kolon varsa, onları at
extra_val_cols  = [c for c in X_val_fe.columns  if c not in X_train_fe.columns]
extra_test_cols = [c for c in X_test_fe.columns if c not in X_train_fe.columns]

if extra_val_cols:
    X_val_fe = X_val_fe.drop(columns=extra_val_cols)
if extra_test_cols:
    X_test_fe = X_test_fe.drop(columns=extra_test_cols)

# Son olarak kolon sıralarını bire bir aynı yap
X_val_fe  = X_val_fe[X_train_fe.columns]
X_test_fe = X_test_fe[X_train_fe.columns]

# 2) Kolon tiplerine göre listeleri hazırla

# Kategorik kolonlar (one-hot)
cat_cols_base = [
    "Gender", "Married", "Education",
    "Self_Employed", "Property_Area",
    "Dependents", "Credit_History"
]
cat_cols_enc = [c for c in cat_cols_base if c in X_train_fe.columns]

# Flag kolonları (0/1, sadece passthrough)
flag_cols_enc = [c for c in missing_flag_cols if c in X_train_fe.columns] #### in flag_cols yerine missing_flag_cols yazıldı

# Sayısal kolonlar (RobustScaler)
num_cols_enc = [
    c for c in X_train_fe.select_dtypes(include=[np.number]).columns
    if c not in flag_cols_enc and c not in cat_cols_enc
]

print("One-Hot yapılacak kategorik kolonlar:", cat_cols_enc)
print("Sayısal (RobustScaler) kolonları:", num_cols_enc)
print("Flag kolonları:", flag_cols_enc)

# 3) ColumnTransformer tanımı

preprocess = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), num_cols_enc),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols_enc),
        ("flag", "passthrough", flag_cols_enc),
    ],
    remainder="drop"
)

# 4) Fit (train) + transform (train/val/test)

X_train_final = preprocess.fit_transform(X_train_fe)
X_val_final   = preprocess.transform(X_val_fe)
X_test_final  = preprocess.transform(X_test_fe)

print("X_train_final shape:", X_train_final.shape)
print("X_val_final shape:",   X_val_final.shape)
print("X_test_final shape:",  X_test_final.shape)

# 5) Feature isimlerini çıkarıp DataFrame'e çevir

# One-hot sonrası kategorik feature isimleri
ohe = preprocess.named_transformers_["cat"]
cat_ohe_names = ohe.get_feature_names_out(cat_cols_enc)

feature_names = (
    list(num_cols_enc) +
    list(cat_ohe_names) +
    list(flag_cols_enc)
)

X_train_final_df = pd.DataFrame(X_train_final, columns=feature_names, index=X_train_fe.index)
X_val_final_df   = pd.DataFrame(X_val_final,   columns=feature_names, index=X_val_fe.index)
X_test_final_df  = pd.DataFrame(X_test_final,  columns=feature_names, index=X_test_fe.index)


One-Hot yapılacak kategorik kolonlar: ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area', 'Dependents', 'Credit_History']
Sayısal (RobustScaler) kolonları: ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Total_Income', 'EMI', 'Income_per_Loan', 'Loan_Term_Years', 'Log_Total_Income', 'Log_LoanAmount', 'Many_Dependents', 'Coapplicant_Income_Ratio', 'Income_to_EMI', 'Per_Capita_Income']
Flag kolonları: ['is_missing_Gender', 'is_missing_Married', 'is_missing_Dependents', 'is_missing_Self_Employed', 'is_missing_LoanAmount', 'is_missing_Loan_Amount_Term', 'is_missing_Credit_History']
X_train_final shape: (491, 38)
X_val_final shape: (123, 38)
X_test_final shape: (367, 38)


## 7) FEATURE SELECTION – Mutual Information (MI)

In [7]:
from sklearn.feature_selection import mutual_info_classif

# 1) Giriş veri setleri (Encoding + Scaling sonrası)
X_train_fs = X_train_final_df.copy()
X_val_fs   = X_val_final_df.copy()
X_test_fs  = X_test_final_df.copy()

y_train_fs = y_train_fe.copy()
y_val_fs   = y_val_fe.copy()

print("X_train_fs shape:", X_train_fs.shape)
print("X_val_fs shape:  ", X_val_fs.shape)
print("X_test_fs shape: ", X_test_fs.shape)

# 2) Mutual Information skorlarını hesapla

mi_scores = mutual_info_classif(
    X_train_fs,
    y_train_fs,
    random_state=42
)

mi_series = pd.Series(mi_scores, index=X_train_fs.columns).sort_values(ascending=False)

print("\n=== Mutual Information Skorları (Büyükten Küçüğe) ===")
print(mi_series)

# İstersen ilk N feature'a bakalım (örneğin top 20)
TOP_N = 20
print(f"\nEn yüksek MI skoruna sahip ilk {TOP_N} değişken:")
print(mi_series.head(TOP_N))

# 3) Eşik / Top-K seçimi ile feature subset oluşturma

# Seçim 1: Top-K feature (örnek: en iyi 25 değişken)
TOP_K = 25
top_k_features = mi_series.head(TOP_K).index.tolist()

print(f"\nSeçilen top-{TOP_K} feature listesi:")
print(top_k_features)

# Alternatif: MI skoruna göre eşik belirleme
# örn: mi_series[mi_series > 0.01].index.tolist()

# 4) Son X matrislerini bu seçilen feature'lara göre daralt

X_train_mi = X_train_fs[top_k_features].copy()
X_val_mi   = X_val_fs[top_k_features].copy()
X_test_mi  = X_test_fs[top_k_features].copy()

print("\nMI sonrası şekiller:")
print("X_train_mi:", X_train_mi.shape)
print("X_val_mi:  ", X_val_mi.shape)
print("X_test_mi: ", X_test_mi.shape)

# 5) (Opsiyonel) MI skorlarını DataFrame olarak kaydet

mi_df = mi_series.reset_index()
mi_df.columns = ["feature", "mi_score"]
mi_df.to_csv("../data/processed/feature_importance_mutual_info.csv", index=False)

X_train = X_train_mi
y_train = y_train_fs

X_val   = X_val_mi
y_val   = y_val_fs

X_test  = X_test_mi

X_train_fs shape:

 (491, 38)
X_val_fs shape:   (123, 38)
X_test_fs shape:  (367, 38)

=== Mutual Information Skorları (Büyükten Küçüğe) ===
Credit_History_1.0             0.128569
Credit_History_0.0             0.109738
is_missing_LoanAmount          0.045006
Married_Yes                    0.035962
Property_Area_Rural            0.020308
CoapplicantIncome              0.017355
is_missing_Credit_History      0.014031
Many_Dependents                0.013767
Coapplicant_Income_Ratio       0.010365
Education_Graduate             0.006973
Married_No                     0.004710
Property_Area_Urban            0.004572
Income_to_EMI                  0.002396
Income_per_Loan                0.000916
Dependents_2.0                 0.000376
is_missing_Married             0.000000
is_missing_Gender              0.000000
is_missing_Dependents          0.000000
Dependents_3.0                 0.000000
is_missing_Self_Employed       0.000000
is_missing_Loan_Amount_Term    0.000000
Dependents_1.0                 0.00000

In [8]:
import json
import os

# Klasör var mı kontrol et, yoksa oluştur
os.makedirs("../data/processed", exist_ok=True)

# 1) Feature matrislerini kaydet

# Modeling notebook'da beklenen isimlere göre kaydediyoruz
# old ->>> X_train_mi.to_csv("../data/processed/X_train_smote.csv", index=False)
X_train_mi.to_csv("../data/processed/X_train_selected.csv", index=False)

X_val_mi.to_csv("../data/processed/X_val_encoded.csv", index=False)
X_test_mi.to_csv("../data/processed/X_test_encoded.csv", index=False)

# 2) Hedef (label) vektörlerini kaydet
pd.DataFrame({"Loan_Status": y_train_final}).to_csv(
    # old ->>>"../data/processed/y_train_smote.csv", index=False
    "../data/processed/y_train_final.csv", index=False
)
pd.DataFrame({"Loan_Status": y_val_final}).to_csv(
    "../data/processed/y_val_final.csv", index=False
)

print("✓ X_train_selected, y_train_final, X_val_encoded, y_val_final, X_test_encoded kaydedildi.")

# 3) Seçilmiş feature listesini ayrıca json olarak kaydet
final_feature_list = list(X_train_mi.columns)

with open("../data/processed/final_feature_list.json", "w", encoding="utf-8") as f:
    json.dump(final_feature_list, f, ensure_ascii=False, indent=2)

print("✓ final_feature_list.json kaydedildi.")


✓ X_train_selected, y_train_final, X_val_encoded, y_val_final, X_test_encoded kaydedildi.
✓ final_feature_list.json kaydedildi.


In [9]:
from imblearn.over_sampling import SMOTE
import json
import os

# Klasör var mı kontrol et, yoksa oluştur
os.makedirs("../data/processed", exist_ok=True)

# A) SMOTE Uygulama (Sadece Train Setine!)
print(f"SMOTE öncesi sınıf dağılımı (Train): {y_train_fs.value_counts().to_dict()}")

# SMOTE nesnesi
smote = SMOTE(random_state=42)

# X_train_mi (Feature Selection yapılmış hali) üzerinde SMOTE uygula
X_train_smote, y_train_smote = smote.fit_resample(X_train_mi, y_train_fs)

# SMOTE çıktısı numpy array olabilir, DataFrame'e geri çevirelim (Feature isimlerini koruyarak)
X_train_smote = pd.DataFrame(X_train_smote, columns=X_train_mi.columns)
y_train_smote = pd.Series(y_train_smote, name="Loan_Status")

print(f"SMOTE sonrası sınıf dağılımı (Train): {y_train_smote.value_counts().to_dict()}")

# B) Dosyaları Kaydetme

# 1. Normal (Dengesiz - Orijinal) Eğitim Seti
X_train_mi.to_csv("../data/processed/X_train_selected.csv", index=False)
y_train_fs.to_frame("Loan_Status").to_csv("../data/processed/y_train_final.csv", index=False)

# 2. SMOTE Uygulanmış (Dengeli) Eğitim Seti
X_train_smote.to_csv("../data/processed/X_train_smote.csv", index=False)
y_train_smote.to_frame("Loan_Status").to_csv("../data/processed/y_train_smote.csv", index=False)

# 3. Validation ve Test Setleri (Asla SMOTE uygulanmaz!)
X_val_mi.to_csv("../data/processed/X_val_encoded.csv", index=False)
y_val_fs.to_frame("Loan_Status").to_csv("../data/processed/y_val_final.csv", index=False)

X_test_mi.to_csv("../data/processed/X_test_encoded.csv", index=False)
# Test setinin label'ı (y_test) yarışma formatında olduğu için genelde yoktur veya ayrı tutulur.
# Eğer elinde y_test varsa onu da kaydedebilirsin.

print("\n✓ Dosyalar başarıyla kaydedildi:")
print("  - Normal Train: X_train_selected.csv & y_train_final.csv")
print("  - SMOTE Train:  X_train_smote.csv & y_train_smote.csv")
print("  - Validation:   X_val_encoded.csv & y_val_final.csv")
print("  - Test:         X_test_encoded.csv")

# C) Metadata Kaydı
final_feature_list = list(X_train_mi.columns)

with open("../data/processed/final_feature_list.json", "w", encoding="utf-8") as f:
    json.dump(final_feature_list, f, ensure_ascii=False, indent=2)

print("✓ final_feature_list.json kaydedildi.")

SMOTE öncesi sınıf dağılımı (Train): {1: 337, 0: 154}
SMOTE sonrası sınıf dağılımı (Train): {1: 337, 0: 337}

✓ Dosyalar başarıyla kaydedildi:
  - Normal Train: X_train_selected.csv & y_train_final.csv
  - SMOTE Train:  X_train_smote.csv & y_train_smote.csv
  - Validation:   X_val_encoded.csv & y_val_final.csv
  - Test:         X_test_encoded.csv
✓ final_feature_list.json kaydedildi.
